In [7]:
# delete files from previous runs
!rm -f concrete_data*

# download the file containing the data in CSV format
!wget https://cocl.us/concrete_data


--2020-03-08 07:59:40--  https://cocl.us/concrete_data
Resolving cocl.us (cocl.us)... 158.85.108.83, 169.48.113.194, 158.85.108.86
Connecting to cocl.us (cocl.us)|158.85.108.83|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv [following]
--2020-03-08 07:59:41--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58988 (58K) [text/csv]
Saving to: ‘concrete_data’

100%[======================================>] 58,988      --.-K/s   in 0.003s  

2020-03-08 07:59:41 (17.

In [53]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import keras
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
concrete_data = pd.read_csv('concrete_data')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


Regression model

In [54]:
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

Normalized version of the data

In [57]:
concrete_data_columns = concrete_data.columns
predictors  = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']]
n_cols = predictors.shape[1]
normalised = (predictors - predictors.mean()) / predictors.std()
normalised.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


Splitting Data 27 % and sorting data

In [58]:
X_train, X_test, y_train, y_test = train_test_split(normalised, aim, test_size=0.27, random_state=42)

Training the network

In [63]:
model = regression_model()
epochs = 100
model.fit(X_train, y_train, epochs=epochs, verbose=1)

Epoch 1/100
751/751 [==============================] - 1s 976us/step - loss: 1551.9286
Epoch 2/100
751/751 [==============================] - 0s 138us/step - loss: 1536.7174
Epoch 3/100
751/751 [==============================] - 0s 141us/step - loss: 1521.5756
Epoch 4/100
751/751 [==============================] - 0s 166us/step - loss: 1505.8312
Epoch 5/100
751/751 [==============================] - 0s 130us/step - loss: 1489.6413
Epoch 6/100
751/751 [==============================] - 0s 163us/step - loss: 1472.2736
Epoch 7/100
751/751 [==============================] - 0s 126us/step - loss: 1454.1846
Epoch 8/100
751/751 [==============================] - 0s 156us/step - loss: 1434.8996
Epoch 9/100
751/751 [==============================] - 0s 137us/step - loss: 1414.6199
Epoch 10/100
751/751 [==============================] - 0s 152us/step - loss: 1393.4279
Epoch 11/100
751/751 [==============================] - 0s 141us/step - loss: 1370.7948
Epoch 12/100
751/751 [===================

Loss value

In [64]:
loss_value = model.evaluate(X_test, y_test)
y_pred = model.predict(X_test)
loss_value

279/279 [==============================] - 0s 473us/step


167.61163969962828

Mean Squared Error

In [65]:
mean_square_error = mean_squared_error(y_test, y_pred)
mean = np.mean(mean_square_error)
standard_deviation = np.std(mean_square_error)
print(mean, standard_deviation)



167.6116409106303 0.0


Total Mean squared error

In [68]:
epochs = 100
mean_squared_errors = []
total_MSE = 50
for i in range(0, total_MSE):
    X_train, X_test, y_train, y_test = train_test_split(normalised, target, test_size=0.27, random_state=i)
    model.fit(X_train, y_train, epochs=epochs, verbose=0)
    MSE = model.evaluate(X_test, y_test, verbose=0)
    print("Mean Squared Error "+str(i+1)+": "+str(MSE))
    y_pred = model.predict(X_test)
    single_mean_square_error = mean_squared_error(y_test, y_pred)
    mean_squared_errors.append(single_mean_square_error)

mean_squared_errors = np.array(mean_squared_errors)
mean = np.mean(mean_squared_errors)
standard_deviation = np.std(mean_squared_errors)

print('\n')
print("Mean: "+str(mean))
print("Standard Deviation: "+str(standard_deviation))

Mean Squared Error 1: 33.138736369362014
Mean Squared Error 2: 38.92086293620448
Mean Squared Error 3: 31.441074241446763
Mean Squared Error 4: 36.07026080333204
Mean Squared Error 5: 34.42662971250473
Mean Squared Error 6: 38.13716578056308
Mean Squared Error 7: 37.36737480300302
Mean Squared Error 8: 30.74433356117604
Mean Squared Error 9: 32.7860962925846
Mean Squared Error 10: 31.296960974252354
Mean Squared Error 11: 33.37509465644864
Mean Squared Error 12: 32.88429319046732
Mean Squared Error 13: 39.742527103765894
Mean Squared Error 14: 37.48517711871841
Mean Squared Error 15: 33.35731737417132
Mean Squared Error 16: 27.699167060168413
Mean Squared Error 17: 36.38098048822
Mean Squared Error 18: 36.31975944409661
Mean Squared Error 19: 33.454317892751384
Mean Squared Error 20: 33.98003431887609
Mean Squared Error 21: 30.682467942596762
Mean Squared Error 22: 33.688062920792554
Mean Squared Error 23: 30.181556551259906
Mean Squared Error 24: 31.318659375645353
Mean Squared Error 

In [69]:
epochs = 50
mean_squared_errors = []
total_MSE = 50
for i in range(0, total_MSE):
    X_train, X_test, y_train, y_test = train_test_split(normalised, target, test_size=0.27, random_state=i)
    model.fit(X_train, y_train, epochs=epochs, verbose=0)
    MSE = model.evaluate(X_test, y_test, verbose=0)
    print("Mean Squared Error "+str(i+1)+": "+str(MSE))
    y_pred = model.predict(X_test)
    single_mean_square_error = mean_squared_error(y_test, y_pred)
    mean_squared_errors.append(single_mean_square_error)

mean_squared_errors = np.array(mean_squared_errors)
mean = np.mean(mean_squared_errors)
standard_deviation = np.std(mean_squared_errors)

print('\n')
print("Mean: "+str(mean))
print("Standard Deviation: "+str(standard_deviation))

Mean Squared Error 1: 31.099665925494232
Mean Squared Error 2: 36.3495901032588
Mean Squared Error 3: 30.07127659295195
Mean Squared Error 4: 34.14864130601234
Mean Squared Error 5: 32.915703927316976
Mean Squared Error 6: 35.830403126268834
Mean Squared Error 7: 35.58189892512496
Mean Squared Error 8: 28.967158765348483
Mean Squared Error 9: 31.70444081077439
Mean Squared Error 10: 30.109061032640465
Mean Squared Error 11: 32.96522575296382
Mean Squared Error 12: 31.460961345276097
Mean Squared Error 13: 37.571396872134194
Mean Squared Error 14: 35.4295412289199
Mean Squared Error 15: 32.13283923022636
Mean Squared Error 16: 26.588960743292258
Mean Squared Error 17: 34.753938531362884
Mean Squared Error 18: 35.04564923795748
Mean Squared Error 19: 31.161770783017612
Mean Squared Error 20: 32.742146399713334
Mean Squared Error 21: 29.753250327161563
Mean Squared Error 22: 33.07115591725996
Mean Squared Error 23: 28.65920830456587
Mean Squared Error 24: 30.405342758342783
Mean Squared E

Epoch: 50 
Mean Squared Error 50: 36.165951322056486
Mean: 32.90620093667656
Standard Deviation: 2.8779024312898267

Epoch: 100
Mean Squared Error 50: 37.43307485546262
Mean: 34.22158076328186
Standard Deviation: 3.0390252925296384